In [1]:
! pip install konlpy

     |████████████████████████████████| 19.4 MB 40.3 MB/s 
     |████████████████████████████████| 448 kB 32.4 MB/s 
     |████████████████████████████████| 86 kB 4.0 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Seq2seq 신경망 구현 및 학습

In [3]:
import random
import tensorflow as tf
from konlpy.tag import Okt

## 하이퍼 파라미터

In [4]:
EPOCHS = 200
NUM_WORDS = 5000

## Encoder

In [5]:
class Encoder(tf.keras.Model):
    def __init__(self):
        super(Encoder, self).__init__()
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 64) 
        # 5000개의 단어를 64크기의 vector로 embedding
        
        self.lstm = tf.keras.layers.LSTM(512, return_state=True)
        '''
          return_state : return하는 output에 최근의 state를 더해주냐에 대한 옵션 
                         즉, hidden state와 cell state를 출력해주기 위한 옵션임 
        '''

    def call(self, x, training=False, mask=None):
        x = self.emb(x)
        _, h, c = self.lstm(x)
        return h, c

## Decoder

In [6]:
class Decoder(tf.keras.Model):
    def __init__(self):
        super(Decoder, self).__init__()
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 64)
        self.lstm = tf.keras.layers.LSTM(512, return_sequences=True, return_state=True)
        '''
        return_sequences : return할 output을 full sequence 또는 마지막에만 출력할지 결정하는 옵션
        - true : full sequence 출력
        - false : 마지막에만 출력
        '''
        self.dense = tf.keras.layers.Dense(NUM_WORDS, activation='softmax')

    def call(self, inputs, training=False, mask=None):
        x, h, c = inputs
        x = self.emb(x)
        x, h, c = self.lstm(x, initial_state=[h, c])
        # initial_state : 셀의 첫번쨰 호출로 전달될 초기 상태의 텐서 목록
        return self.dense(x), h, c

## Seq2seq

In [7]:
class Seq2seq(tf.keras.Model):
    def __init__(self, sos, eos):
        super(Seq2seq, self).__init__()
        self.enc = Encoder()
        self.dec = Decoder()
        self.sos = sos
        self.eos = eos

    def call(self, inputs, training=False, mask=None):
        if training is True:
            x, y = inputs
            h, c = self.enc(x)
            y, _, _ = self.dec((y, h, c))
            # shifted output, hidden state, cell state을 초기값으로 입력 받음
            # y : 전체 문장
            return y
        else:
            x = inputs
            h, c = self.enc(x)
            y = tf.convert_to_tensor(self.sos)
            # decoder 단에 제일 먼저 sos를 넣어주게끔 tensor화 시킴
            y = tf.reshape(y, (1, 1))

            seq = tf.TensorArray(tf.int32, 64)

            for idx in tf.range(64):
                y, h, c = self.dec([y, h, c])
                y = tf.cast(tf.argmax(y, axis=-1), dtype=tf.int32)
                '''
                예측한값을 다시 다음 step의 입력으로 넣어주기 위한 작업
                : 위의 출력으로 나온 y는 softmax를 지나서 나온 값이므로 가장 높은 값의 index값을 tf.int32로 형변환해주고
                TensorArray seq의 idx에 y를 추가해준다.
                '''

                y = tf.reshape(y, (1, 1))
                # 실제로 네트워크를 사용할 때 Batch를 고려해서 사용해야 하기 때문에 (1,1)으로 설정해 준다.

                seq = seq.write(idx, y)

                if y == self.eos:
                    break

            return tf.reshape(seq.stack(), (1, 64))

## 학습, 테스트 루프 정의

In [8]:
# Implement training loop
@tf.function
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_accuracy):
    output_labels = labels[:, 1:]
    shifted_labels = labels[:, :-1]
    with tf.GradientTape() as tape:
        predictions = model([inputs, shifted_labels], training=True)
        loss = loss_object(output_labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(output_labels, predictions)

# Implement algorithm test
@tf.function
def test_step(model, inputs):
    return model(inputs, training=False)

## 데이터셋 준비


In [10]:
dataset_file = '/content/drive/MyDrive/패스트캠퍼스/Part4) 딥러닝 3 STEP의 기초/dataset/chatbot_data.csv'  # acquired from 'http://www.aihub.or.kr' and modified
okt = Okt()

with open(dataset_file, 'r') as file:
    lines = file.readlines()
    seq = [' '.join(okt.morphs(line)) for line in lines]

questions = seq[::2]
answers = ['\t ' + lines for lines in seq[1::2]]

num_sample = len(questions)

perm = list(range(num_sample))
random.seed(0)
random.shuffle(perm)

train_q = list()
train_a = list()
test_q = list()
test_a = list()

for idx, qna in enumerate(zip(questions, answers)):
    q, a = qna
    if perm[idx] > num_sample//5:
        train_q.append(q)
        train_a.append(a)
    else:
        test_q.append(q)
        test_a.append(a)

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=NUM_WORDS,
                                                  filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~')
# filters의 default에는 \t,\n도 제거하기 때문에 이 둘을 제외하고 나머지 문장기호들만 제거하게끔 변경해주었다.

tokenizer.fit_on_texts(train_q + train_a)

train_q_seq = tokenizer.texts_to_sequences(train_q)
train_a_seq = tokenizer.texts_to_sequences(train_a)

test_q_seq = tokenizer.texts_to_sequences(test_q)
test_a_seq = tokenizer.texts_to_sequences(test_a)

x_train = tf.keras.preprocessing.sequence.pad_sequences(train_q_seq,
                                                        value=0,
                                                        padding='pre',
                                                        maxlen=64)
y_train = tf.keras.preprocessing.sequence.pad_sequences(train_a_seq,
                                                        value=0,
                                                        padding='post',
                                                        maxlen=65)
'''
y값에는 maxlen=65인 이유는 앞에 SOS와 뒤에 EOS가 붙어 있기 때문
학습시에는 앞에 하나를 떼므로 실제로는 64길이만 사용하는 것과 동일하게 된다.
'''

x_test = tf.keras.preprocessing.sequence.pad_sequences(test_q_seq,
                                                       value=0,
                                                       padding='pre',
                                                       maxlen=64)
y_test = tf.keras.preprocessing.sequence.pad_sequences(test_a_seq,
                                                       value=0,
                                                       padding='post',
                                                       maxlen=65)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(1024)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(1).prefetch(1024)

## 학습 환경 정의
### 모델 생성, 손실함수, 최적화 알고리즘, 평가지표 정의

In [11]:
# Create model
model = Seq2seq(sos=tokenizer.word_index['\t'],
                eos=tokenizer.word_index['\n'])

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

## 학습 루프 동작

In [12]:
for epoch in range(EPOCHS):
    for seqs, labels in train_ds:
        train_step(model, seqs, labels, loss_object, optimizer, train_loss, train_accuracy)

    template = 'Epoch {}, Loss: {}, Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100))

    train_loss.reset_states()
    train_accuracy.reset_states()

Epoch 1, Loss: 4.106929302215576, Accuracy: 83.13361358642578
Epoch 2, Loss: 0.6438242793083191, Accuracy: 90.3900375366211
Epoch 3, Loss: 0.5867494344711304, Accuracy: 90.54667663574219
Epoch 4, Loss: 0.5632861256599426, Accuracy: 90.96961212158203
Epoch 5, Loss: 0.550795316696167, Accuracy: 91.16932678222656
Epoch 6, Loss: 0.5446475148200989, Accuracy: 91.13016510009766
Epoch 7, Loss: 0.5416964888572693, Accuracy: 91.10275268554688
Epoch 8, Loss: 0.5380862355232239, Accuracy: 91.09883880615234
Epoch 9, Loss: 0.5295473337173462, Accuracy: 91.14191436767578
Epoch 10, Loss: 0.5338311195373535, Accuracy: 91.09100341796875
Epoch 11, Loss: 0.5234279036521912, Accuracy: 91.10667419433594
Epoch 12, Loss: 0.5199558734893799, Accuracy: 91.14191436767578
Epoch 13, Loss: 0.5169908404350281, Accuracy: 91.21240234375
Epoch 14, Loss: 0.5063393712043762, Accuracy: 91.22806549072266
Epoch 15, Loss: 0.4985678493976593, Accuracy: 91.28681182861328
Epoch 16, Loss: 0.48776867985725403, Accuracy: 91.36904

## 테스트 루프

In [13]:
for test_seq, test_labels in test_ds:
    prediction = test_step(model, test_seq)
    test_text = tokenizer.sequences_to_texts(test_seq.numpy())
    gt_text = tokenizer.sequences_to_texts(test_labels.numpy())
    texts = tokenizer.sequences_to_texts(prediction.numpy())
    print('_')
    print('q: ', test_text)
    print('a: ', gt_text)
    print('p: ', texts)

_
q:  ['여기 기프티콘 되죠 \n']
a:  ['\t 네 현금영수증 해드릴까 요 \n']
p:  ['할인 카드 도 안 하시나요 \n']
_
q:  ['네 에 테이크 아웃 도 가능한가요 \n']
a:  ['\t 네 로 오시 면 테이크 아웃 잔 에 담아 드려요 \n']
p:  ['네 다음 에 써도 됩니다 \n']
_
q:  ['아메리카노 톨 사이즈 로 주세요 \n']
a:  ['\t 따뜻한 거 로 드릴 까요 \n']
p:  ['드시고 가시나요 \n']
_
q:  ['진동 을 따로 주시나요 \n']
a:  ['\t 주 번호 로 드리겠습니다 \n']
p:  ['네 드릴게요 잠시 만 요 \n']
_
q:  ['자리 있나요 \n']
a:  ['\t 네 있습니다 \n']
p:  ['네 자몽 차는 있습니다 \n']
_
q:  ['그럼 루이보스 밀크 티 하나 \n']
a:  ['\t 네 알겠습니다 \n']
p:  ['여기 사이즈 는 지금 없어요 \n']
_
q:  ['다음 에 무료 로 하고 엔 도장 찍어주세요 \n']
a:  ['\t 네 \n']
p:  ['네 이리 주세요 \n']
_
q:  ['아메리카노 한 잔 에 얼마 죠 \n']
a:  ['\t 입니다 \n']
p:  ['4000원 입니다 \n']
_
q:  ['얼마나 \n']
a:  ['\t 바로 만들어 드릴게요 \n']
p:  ['네 진동 벨 울리면 픽업 테이블 로 와주세요 \n']
_
q:  ['카푸치노 는 로 주시 고 아메리카노 는 로 \n']
a:  ['\t 네 더 없으세요 \n']
p:  ['할인 적용 해서 9300원 결제 도 와 드릴게요 \n']
_
q:  ['아메리카노 는 어떤 종류 가 있나요 \n']
a:  ['\t 디카 페인 과 기본 아메리카노 2 종류 있습니다 \n']
p:  ['헤이즐넛 시럽 은 있습니다 \n']
_
q:  ['카카오 페이 로 결제 가능한가요 \n']
a:  ['\t 네 가능합니다 \n']
p:  ['네 \n']
_
q:  ['오늘 의 커피 는 커피 로 하나요 맛 이 \n']
a